In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json


In [2]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('GEMINI_API_KEY')


In [3]:
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key = OPENAI_API_KEY)

In [4]:
response = client.chat.completions.create(model = "gemini-2.5-flash",messages={'role':'user','content':'hellow'})

In [5]:
print(response.choices[0].message.content)

Hello! How can I help you today?


In [6]:
data_base = {'karachi to lahore':'plane name = Pk.2003 ,date=21 feb ,time = 11am','lahore to karachi':"plane name = pk.3000, date=22 feb,time=9am"}
def flight_detailes(Travelling):
    print(f'Tool calling for {Travelling}')
    plane_name = data_base.get(Travelling['Travelling'],"Unknow ticket")
    return f"Your travelling plane details is {plane_name}"



In [7]:
#funcation for booking 
def book_ticket(Travelling):
    print(f'Processing for booking ticket for {Travelling}')
    for key , value in data_base.items():
        if key == Travelling:
            #print("Your Ticket is booked")
            return f"your ticket booked and details of ticket is {data_base[Travelling]}"

In [52]:
#Defining the function to cancel the booked ticket 
def ticket_cancel(Travelling):
    print("Checking database for ticket to cancel")
    for key , val in data_base.items():
        if key == Travelling:
            return f"You are ticket from {Travelling} is cancelled !"

In [53]:
ticket_cancel('lahore to karachi')

Checking database for ticket to cancel


'You are ticket from lahore to karachi is cancelled !'

In [8]:
print(book_ticket('karachi to lahore'))

Processing for booking ticket for karachi to lahore
your ticket booked and details of ticket is plane name = Pk.2003 ,date=21 feb ,time = 11am


In [29]:
tool1 = {
    'name':'flight_detailes',
    'description':'This funcation provide you the name of travelling plane',
    'parameters':{
        'type':'object',
        "properties":{
            'Travelling':{
                'type':'string',
                'description': "This is required parameter for getting the travelling plane name "
            },
        },
        'required':['Travelling']
    }
    
}

In [20]:
tool2 = {
    "name":'book_ticket',
    "description":"User this tool when user went to booked the ticket",
    'parameters':{
        'type':'object',
        "properties":{
            'Travelling':{
                'type':'string',
                'description':'This is required parameter for booking the plane'
            },
        },
        'required':['Travelling']
    }
}

In [54]:
tool3 = {
    "name":'ticket_cancel',
    'description':'Call this funcation when user went to cancel their booked ticket',
    'parameters':{
        'type':'object',
        'properties':{
            'Travelling':{
                'type':'string',
                'description':'This required parameters for cancelling the ticket'
            },
        }
    },
    'required': ['Travelling']
}

In [55]:
#tool = tool1
tool = [tool1 ,tool2,tool3]

In [31]:
tools = {'type':'function','function':tool}

In [50]:
system_message = """ You are an intelliegent Airport helper Agent  your tasked is to reply the user base on their input :
input are of three type: 
1st type = user : I went to travelled from karachi to lahore
assitent : reply wih plane name , date and time of avalibale plane
example:1 i went to go from islamabad to karachi?


answer : plane name = pk 1000 takeoff 20 feb at 9 pm 

example:2 i went to go from lahore to islamabad? 
answer: plane name pk 60000 takeoff 29 feb at 10 pm

2nd type = user : I went to travelled from lahore to karachi book the ticket for me
assistent : Congratulation i will  book the ticket for you , plane name pk.9000 , date = 20 feb , time =7am


3nd type = user : I Went to cancel the ticket that i booked for karachi to lahore 
assistent : Congratulation Your ticket is cancel

 """

In [56]:
def handle_tool(content):
    content1 = content.tool_calls[0]
    if content1.function.name == 'flight_detailes':
        arg = json.loads(content1.function.arguments)
        result = flight_detailes(arg)
        response ={
            'role':'tool',
            'content':result,
            "tool_call_id": content1.id
        }
    if content1.function.name == 'book_ticket':
        arg = json.loads(content1.function.arguments)
        result = book_ticket(arg)
        response = {'role':'tool','content':result,'tool_call_id':content1.id}

    if content1.function.name == 'ticket_cancel':
        args = json.loads(content1.function.arguments)
        result = ticket_cancel(args) 
        response = {'role':'tool','content':result,'tool_call_id':content1.id} 
          

    return response    
    

In [57]:
def chat_with_ai(message,history):
    history = [{'role':h['role'],'content':h['content']}for h in history]
    message = [{'role':'system','content':system_message}] + history + [{"role":'user',"content":message}]
    response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)

    if response.choices[0].finish_reason=='tool_calls': 
        content = response.choices[0].message
        tool_call1 = content.tool_calls[0]
        if tool_call1.function.name == 'flight_detailes':
            print('Tool calling for Flight detailes')
            result = handle_tool(content)
            print('Coming back to llm with fligh details')
            message.append(content)
            message.append(result)
            print('Second call to LLm')
            response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)
        if tool_call1.function.name == "book_ticket":
            print('Tool calling for the booking ticket')
            result = handle_tool(content)
            print('Comming back after booking ticket')
            message.append(content)
            message.append(result)
            print('Making the second time call')
            response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)
        if tool_call1.function.name == 'ticket_cancel':
            print('Calling the tool for cancelling the ticket')
            result = handle_tool(content)        
            print('Coming back after cancelling ticket')
            message.append(content)
            message.append(result)
            response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)

    return response.choices[0].message.content    

        
    

In [35]:
import gradio as gr

In [58]:
gr.ChatInterface(fn=chat_with_ai,type='messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7872
* Running on public URL: https://78e38046387d7e8b66.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
